In [1]:
import pandas as pd
import numpy as np
from pykospacing import Spacing
from konlpy.tag import Okt
from string import punctuation
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

In [2]:
data = pd.read_csv('ko_train_label.csv')
data = data.drop(labels = 'id', axis = 1)
data = data.dropna(how = 'any', axis = 0)
data = data.drop_duplicates(subset = ['document'])
data

,document,toxic,obscene,threat,insult,identity_hate
0,아 더빙.. 진짜 짜증나네요 목소리,1,0,0,0,0
1,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,1,0,0,0,0
2,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,1,0,0,0,0
3,원작의 긴장감을 제대로 살려내지못했다.,1,0,0,0,0
4,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,1,0,0,0,1
...,...,...,...,...,...,...
9994,혹시나 그래도 카메론디아즈니까 하고봤는데...먼이런영화를..이도저도아닌 .아암튼 시...,1,0,0,0,0
9995,10점주는것들 한국영화는 1점주네. M창,1,0,0,0,0
9996,영상도 아름답고 뭘 말하는지 알겠지만 그렇기 때문에 짜증나고 답답하다,1,0,0,0,0
9997,영화를 왜 영화라고 하는지 모르는 애들이 나왔네.,1,0,0,0,0


In [3]:
st = set(['은', '는', '이', '가', '을', '를', '에', '에게', '그', '저', '저희', '우리', '더', '나', '로', 
      '영화', '감독', '안', '다', '것', '연기', '한', '진짜'])
p = list(punctuation)
st.update(p)

In [4]:
data['document'] = data['document'].str.replace('[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]', '')
data['document'] = data['document'].str.replace('^ +', '')
data['document'] = data['document'].replace(' ', np.nan)
data['document'].dropna(how = 'any', axis = 0)
data

C:\Users\김철용\AppData\Local\Temp/ipykernel_17460/119940063.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['document'] = data['document'].str.replace('[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]', '')
C:\Users\김철용\AppData\Local\Temp/ipykernel_17460/119940063.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data['document'] = data['document'].str.replace('^ +', '')


,document,toxic,obscene,threat,insult,identity_hate
0,아 더빙 진짜 짜증나네요 목소리,1,0,0,0,0
1,교도소 이야기구먼 솔직히 재미는 없다평점 조정,1,0,0,0,0
2,막 걸음마 뗀 세부터 초등학교 학년생인 살용영화ㅋㅋㅋ별반개도 아까움,1,0,0,0,0
3,원작의 긴장감을 제대로 살려내지못했다,1,0,0,0,0
4,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지정말 발로해도 그것보단 낫...,1,0,0,0,1
...,...,...,...,...,...,...
9994,혹시나 그래도 카메론디아즈니까 하고봤는데먼이런영화를이도저도아닌 아암튼 시간 남아도보...,1,0,0,0,0
9995,점주는것들 한국영화는 점주네 창,1,0,0,0,0
9996,영상도 아름답고 뭘 말하는지 알겠지만 그렇기 때문에 짜증나고 답답하다,1,0,0,0,0
9997,영화를 왜 영화라고 하는지 모르는 애들이 나왔네,1,0,0,0,0


In [5]:
spacing = Spacing()
s_sl = []
for sentence in data['document']:
    s_sl.append(spacing(sentence))
data['document'] = s_sl

In [6]:
okt = Okt()
okt_l = []
for sentence in data['document']:
    okt_l.append(okt.morphs(sentence))
okt_l

[['아', '더빙', '진짜', '짜증나네요', '목소리'],
 ['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정'],
 ['막',
  '걸음',
  '마',
  '뗀',
  '세',
  '부터',
  '초등학교',
  '학년',
  '생인',
  '살용',
  '영화',
  'ㅋㅋㅋ',
  '별반',
  '개도',
  '아까',
  '움'],
 ['원작', '의', '긴장감', '을', '제대로', '살려내지', '못', '했다'],
 ['별',
  '반개',
  '도',
  '아깝다',
  '욕',
  '나온',
  '다',
  '이',
  '응경',
  '길용우',
  '연',
  '기',
  '생활',
  '이',
  '몇',
  '년',
  '인지',
  '정말',
  '발',
  '로',
  '해도',
  '그것',
  '보단',
  '낫겟다',
  '납치',
  '감금',
  '만',
  '반복',
  '반복',
  '이',
  '드라마',
  '는',
  '가족',
  '도',
  '없다',
  '연기',
  '못',
  '하는',
  '사람',
  '만',
  '모',
  '엿',
  '네'],
 ['울면',
  '서',
  '손',
  '들고',
  '횡단보도',
  '건널',
  '때',
  '뛰',
  '쳐',
  '나올',
  '뻔',
  '이',
  '범수',
  '연기',
  '드럽게',
  '못',
  '해'],
 ['취향',
  '은',
  '존중',
  '한',
  '다지',
  '만',
  '진짜',
  '내',
  '생',
  '에',
  '극장',
  '에서',
  '본',
  '영화',
  '중',
  '가장',
  '노잼',
  '노',
  '감동',
  '임',
  '스토리',
  '도',
  '어거지',
  '고',
  '감동',
  '도',
  '어거지'],
 ['굿바이',
  '레닌',
  '표절',
  '인',
  '것',
  '은',
  '이해',
  '하는데

In [24]:
st_okt_l = []
for sentence in okt_l:
    st_okt_l.append([word for word in sentence if word not in st])

[['아', '더빙', '짜증나네요', '목소리'],
 ['교도소', '이야기', '구먼', '솔직히', '재미', '없다', '평점', '조정'],
 ['막',
  '걸음',
  '마',
  '뗀',
  '세',
  '부터',
  '초등학교',
  '학년',
  '생인',
  '살용',
  'ㅋㅋㅋ',
  '별반',
  '개도',
  '아까',
  '움'],
 ['원작', '의', '긴장감', '제대로', '살려내지', '못', '했다'],
 ['별',
  '반개',
  '도',
  '아깝다',
  '욕',
  '나온',
  '응경',
  '길용우',
  '연',
  '기',
  '생활',
  '몇',
  '년',
  '인지',
  '정말',
  '발',
  '해도',
  '그것',
  '보단',
  '낫겟다',
  '납치',
  '감금',
  '만',
  '반복',
  '반복',
  '드라마',
  '가족',
  '도',
  '없다',
  '못',
  '하는',
  '사람',
  '만',
  '모',
  '엿',
  '네'],
 ['울면',
  '서',
  '손',
  '들고',
  '횡단보도',
  '건널',
  '때',
  '뛰',
  '쳐',
  '나올',
  '뻔',
  '범수',
  '드럽게',
  '못',
  '해'],
 ['취향',
  '존중',
  '다지',
  '만',
  '내',
  '생',
  '극장',
  '에서',
  '본',
  '중',
  '가장',
  '노잼',
  '노',
  '감동',
  '임',
  '스토리',
  '도',
  '어거지',
  '고',
  '감동',
  '도',
  '어거지'],
 ['굿바이', '레닌', '표절', '인', '이해', '하는데', '왜', '뒤', '갈수록', '재미', '없어지냐'],
 ['재미없다',
  '지루하고',
  '같은',
  '음식',
  '인데',
  '도',
  '바베트',
  '의',
  '만찬',
  '하고',
  '넘',
  '차',
  '남바베트',
  '의',
 

In [8]:
t_data = data[['toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values
t_data

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]], dtype=int64)

In [9]:
X = st_okt_l
Y = t_data

In [10]:
len(X), len(Y)

(9928, 9928)

In [11]:
x_data, tt_x, y_data, tt_y = train_test_split(X, Y, test_size = 0.3, random_state = 22)
t_x, v_x, t_y, v_y = train_test_split(x_data, y_data, test_size = 0.2, random_state = 22)

In [12]:
tk = Tokenizer()
tk.fit_on_texts(t_x)
len(tk.word_index)

13046

In [13]:
word_num = len([w for w in sorted(list(tk.word_counts.items()), key = lambda x: x[1]) if w[1] > 4]) + 1
word_num

2002

In [14]:
tk = Tokenizer(num_words = word_num)
tk.fit_on_texts(t_x)

In [15]:
t_t_x = tk.texts_to_sequences(t_x)
t_v_x = tk.texts_to_sequences(v_x)
t_tt_x = tk.texts_to_sequences(tt_x)

In [16]:
drop_t = [idx for idx, sentence in enumerate(t_t_x) if len(sentence) < 1]
drop_v = [idx for idx, sentence in enumerate(t_v_x) if len(sentence) < 1]
drop_tt = [idx for idx, sentence in enumerate(t_tt_x) if len(sentence) < 1]

In [17]:
t_t_x = np.delete(t_t_x, drop_t, axis = 0)
t_y = np.delete(t_y, drop_t, axis = 0)
t_v_x = np.delete(t_v_x, drop_v, axis = 0)
v_y = np.delete(v_y, drop_v, axis = 0)
t_tt_x = np.delete(t_tt_x, drop_tt, axis = 0)
tt_y = np.delete(tt_y, drop_tt, axis = 0)

C:\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [18]:
p_l = max([len(pad_sequences(t_t_x)[0]), len(pad_sequences(t_t_x)[0]), len(pad_sequences(t_t_x)[0])])

t_x = pad_sequences(t_t_x, maxlen = p_l)
v_x = pad_sequences(t_v_x, maxlen = p_l)
tt_x = pad_sequences(t_tt_x, maxlen = p_l)

In [19]:
t_x.shape, v_x.shape, tt_x.shape, t_y.shape, v_y.shape, tt_y.shape

((5447, 51), (1353, 51), (2920, 51), (5447, 5), (1353, 5), (2920, 5))

In [20]:
preprocessed_data = {}
preprocessed_data['train_x'] = t_x
preprocessed_data['train_y'] = t_y
preprocessed_data['test_x'] = tt_x
preprocessed_data['test_y'] = tt_y
preprocessed_data['validation_x'] = v_x
preprocessed_data['validation_y'] = v_y
preprocessed_data['token'] = tk
preprocessed_data['sentence_len'] = p_l
preprocessed_data['stop_words'] = st
preprocessed_data['word_num'] = word_num

In [21]:
with open('preprocessed_data.pickle', 'wb') as fw:
    pickle.dump(preprocessed_data, fw)